# Aligner et comparer des textes -- approfondissement et exercices

In [11]:
import collatex
import time
import json
import lxml.etree as etree

Nous allons utiliser le fragment d'un poème de Catulle comme support pour tous nos exercices qui appliquent la partie précédente de l'atelier animée par Elisa. Commençons par charger nos textes:

In [43]:
# On va importer les textes: 1
with open("Catullus/TEXT-O1.txt", "r") as input_text:
    # On va supprimer les sauts de ligne qui ne nous intéressent pas pour l'exercice.
    texte_a = input_text.read().replace("\n", " ")

# et 2
with open("Catullus/TEXT-G2.txt", "r") as input_text:
    texte_b = input_text.read().replace("\n", " ")


# et 3
with open("Catullus/TEXT-Bodmer47-1.txt", "r") as input_text:
    texte_c = input_text.read().replace("\n", " ")

On peut imprimer le texte 3 pour vérifier qu'il est bien chargé:

In [44]:
print(text_3)

Passeris appelatio Passer delitiae meae puellae Qui cum ludere quem in sinu tenere Quoi primum digitum dare appententi Atque acris solet incitare morsus Cum desyderio meo nitenti Charum nescio quid libet iocari Insolatiolum sui doloris Credo ut quum gravis acquiescat ardor Tecum ludere sicut ipsa possem Et tristis animi levare curas 


### Deuxième étape: on prépare la collation

On crée une instance Collation de notre librairie collatex, puis on ajoute chacun des textes

In [45]:
collation = collatex.Collation()
collation.add_plain_witness('O1', texte_a)
collation.add_plain_witness('G2', texte_b)
collation.add_plain_witness('Bodmer47', texte_c)

In [46]:
# Regardons le nombre de témoins de l'objet collation:
print(len(collation.witnesses))

3


Commençons par créer une table de collation simple.

In [36]:
result_table = collatex.collate(collation, segmentation=False)
print(result_table)

+----------+----------+-----------+--------+--------+----------+------+---------+-----+-----+--------+------+----+------+--------+------+--------+---------+------+------------+---------+----+-------+-------+----------+--------+-----+-----------+-----+---------+--------+--------+------+-------+--------+--------------+------------+-----+---------+-------+----+------+--------+------------+-------+-------+---------+-------+------+--------+----+---------+-------+--------+-------+
| O1       | -        | -         | -      | Passer | delicie  | mee  | puelle  | Qui | cum | ludere | quem | in | sinu | tenere | Qui  | primum | digitum | dare | at         | petenti | Ea | acris | solet | incitare | morsus | Cum | desiderio | meo | nitenti | Karum  | nescio | quid | libet | iocari | Et           | solaciolum | sui | doloris | Credo | ut | cum  | gravis | acquiescet | ardor | Tecum | luderem | sicut | ipsa | possem | Et | tristis | animi | levare | curas |
| G2       | fletus   | passeris  | lesb

Le résultat est peu lisible car le texte est trop long. Passons à une table d'alignement verticale.

In [37]:
# On collationne avec une table de collation verticale cette fois
result_table = collatex.collate(collation, layout='vertical', segmentation=False)
print(result_table)

+------------+------------+--------------+
|     O1     |     G2     |   Bodmer47   |
+------------+------------+--------------+
|     -      |   fletus   |   Passeris   |
+------------+------------+--------------+
|     -      |  passeris  |  appelatio   |
+------------+------------+--------------+
|     -      |   lesbie   |      -       |
+------------+------------+--------------+
|   Passer   |   Passer   |    Passer    |
+------------+------------+--------------+
|  delicie   |  delicie   |   delitiae   |
+------------+------------+--------------+
|    mee     |    mee     |     meae     |
+------------+------------+--------------+
|   puelle   |   puelle   |   puellae    |
+------------+------------+--------------+
|    Qui     |    Qui     |     Qui      |
+------------+------------+--------------+
|    cum     |    cum     |     cum      |
+------------+------------+--------------+
|   ludere   |   ludere   |    ludere    |
+------------+------------+--------------+
|    quem  

On choisit ici de collationer mot-à-mot, mais une collation plus globale est possible: c'est le paramètre `segmentation` qu'il faut alors changer.

In [38]:
result_table = collatex.collate(collation, layout='vertical', segmentation=True)
print(result_table)

+----------------------+----------------------+----------------------+
|          O1          |          G2          |       Bodmer47       |
+----------------------+----------------------+----------------------+
|          -           |   fletus passeris    |  Passeris appelatio  |
|                      |        lesbie        |                      |
+----------------------+----------------------+----------------------+
|        Passer        |        Passer        |        Passer        |
+----------------------+----------------------+----------------------+
|  delicie mee puelle  |  delicie mee puelle  |    delitiae meae     |
|                      |                      |       puellae        |
+----------------------+----------------------+----------------------+
| Qui cum ludere quem  | Qui cum ludere quem  | Qui cum ludere quem  |
|    in sinu tenere    |    in sinu tenere    |    in sinu tenere    |
+----------------------+----------------------+----------------------+
|     

On peut produire une sortie au format HTML (pages web) qui sera de meilleure facture: c'est fait avec l'argument `output` de notre méthode `collate`. Les options possibles sont `html`, `html2`, `json`, `table`, `xml`, `tei`, `csv`.

Commeçons par la sortie html simple:

In [39]:
# On collationne avec une table de collation verticale cette fois
result_table = collatex.collate(collation, layout='vertical', output="html", segmentation=False)

O1,G2,Bodmer47
-,fletus,Passeris
-,passeris,appelatio
-,lesbie,-
Passer,Passer,Passer
delicie,delicie,delitiae
mee,mee,meae
puelle,puelle,puellae
Qui,Qui,Qui
cum,cum,cum
ludere,ludere,ludere


In [40]:
# On collationne avec une table de collation horizontale
result_table = collatex.collate(collation, layout='horizontal', output="html", segmentation=False)

O1,-,-,-,Passer,delicie,mee,puelle,Qui,cum,ludere,quem,in,sinu,tenere,Qui,primum,digitum,dare,at,petenti,Ea,acris,solet,incitare,morsus,Cum,desiderio,meo,nitenti,Karum,nescio,quid,libet,iocari,Et,solaciolum,sui,doloris,Credo,ut,cum,gravis,acquiescet,ardor,Tecum,luderem,sicut,ipsa,possem,Et,tristis,animi,levare,curas
G2,fletus,passeris,lesbie,Passer,delicie,mee,puelle,Qui,cum,ludere,quem,in,sinu,tenere,Qui,primum,digitum,dare,at,patenti,Et,acris,solet,incitare,morsus,Cum,desiderio,meo,nitenti,Carum,nescio,quid,libet,iocari,Et,solaciolum,sui,doloris,Credo,ut,cum,gravis,adquiescet,ardor,Tecum,luderem,sicut,ipsa,possem,Et,tristis,animi,levare,curas
Bodmer47,Passeris,appelatio,-,Passer,delitiae,meae,puellae,Qui,cum,ludere,quem,in,sinu,tenere,Quoi,primum,digitum,dare,appententi,Atque,-,acris,solet,incitare,morsus,Cum,desyderio,meo,nitenti,Charum,nescio,quid,libet,iocari,Insolatiolum,-,sui,doloris,Credo,ut,quum,gravis,acquiescat,ardor,Tecum,ludere,sicut,ipsa,possem,Et,tristis,animi,levare,curas


On peut aussi proposer aussi une sortie qui mette en valeur les unités d'alignement avec variante formelle par une couleur particulière: c'est l'option `html2`

In [26]:
result_table = collatex.collate(collation, layout='horizontal', output="html2", segmentation=False)
print(result_table)

None


On peut enfin proposer une sortie au format XML, qui pourra ensuite être facilement convertie à une édition TEI.

In [42]:
result_table = collatex.collate(collation, layout='horizontal', output="tei", indent='True', segmentation=False)
print(etree.tostring(etree.fromstring(result_table), pretty_print=True).decode())

<cx:apparatus xmlns:cx="http://interedition.eu/collatex/ns/1.0" xmlns="http://www.tei-c.org/ns/1.0">
	<app>
		<rdg wit="#Bodmer47">Passeris</rdg>
		<rdg wit="#G2">fletus</rdg>
	</app>
	 
	<app>
		<rdg wit="#Bodmer47">appelatio</rdg>
		<rdg wit="#G2">passeris</rdg>
	</app>
	 
	<app>
		<rdg wit="#G2">lesbie</rdg>
	</app>
	 
	Passer
	 
	<app>
		<rdg wit="#Bodmer47">delitiae</rdg>
		<rdg wit="#G2 #O1">delicie</rdg>
	</app>
	 
	<app>
		<rdg wit="#Bodmer47">meae</rdg>
		<rdg wit="#G2 #O1">mee</rdg>
	</app>
	 
	<app>
		<rdg wit="#Bodmer47">puellae</rdg>
		<rdg wit="#G2 #O1">puelle</rdg>
	</app>
	 
	Qui
	 
	cum
	 
	ludere
	 
	quem
	 
	in
	 
	sinu
	 
	tenere
	 
	<app>
		<rdg wit="#Bodmer47">Quoi</rdg>
		<rdg wit="#G2 #O1">Qui</rdg>
	</app>
	 
	primum
	 
	digitum
	 
	dare
	 
	<app>
		<rdg wit="#Bodmer47">appententi</rdg>
		<rdg wit="#G2 #O1">at</rdg>
	</app>
	 
	<app>
		<rdg wit="#Bodmer47">Atque</rdg>
		<rdg wit="#G2">patenti</rdg>
		<rdg wit="#O1">petenti</rdg>
	</app>
	 
	<app>
		<rdg wit="#G